<a href="https://colab.research.google.com/github/BatsaikhanTuguldur/KousenProjects/blob/main/Titanic_survivalrate_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4J 38 　　f20301

# バトサイハントゥグレデル

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# data visualization
import seaborn as sns


# Algorithms
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB


In [ ]:
train_df = pd.read_csv("../input/titanic/train.csv")
test_df = pd.read_csv("../input/titanic/test.csv")
#df_submission = pd.read_csv("../input/titanic/gender_submission.csv")
#df_submission.to_csv('submission.csv', index=False)
train_df.info() #table1
print(train_df.describe()) #table2
train_df.head() #table3

FileNotFoundError: ignored

* ## Holes in the data should be filled, and not relatable datas should be eliminated.  
* ## Training datas be understandable for the computer, meaning datas should be integers.

In [ ]:
#viewing the missing data
total = train_df.isnull().sum().sort_values(ascending=False)
percent_1 = train_df.isnull().sum()/train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)


### PassengerId shall be eliminated due to its lack of connection with survival  
### Ticket has too many different values so it shall be elimintaed as well  
### Even though age has lots of missing values, its strong relationship with survival can be seen from the table2    
### We already have sex feature, so name feature isn't needed too

In [ ]:
#dropping the unnecessary data
train_df = train_df.drop(['PassengerId'], axis=1)
train_df = train_df.drop(['Ticket'], axis=1)
test_df = test_df.drop(['Ticket'], axis=1)
train_df = train_df.drop(['Name'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
print(train_df.info())
print(test_df.info())

### Although cabin has got so many missing values, cabin numbers look like ‘C123’ and the letter refers to the deck.   
### Thus cabin feature shall be dropped after being converted into deck feature. Missing values shall be turned into 0.


In [ ]:
#
import re
deck = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5, "F": 6, "G": 7, "U": 8}
data = [train_df, test_df]

for dataset in data:
    dataset['Cabin'] = dataset['Cabin'].fillna("U0")
    dataset['Deck'] = dataset['Cabin'].map(lambda x: re.compile("([a-zA-Z]+)").search(x).group())
    dataset['Deck'] = dataset['Deck'].map(deck)
    dataset['Deck'] = dataset['Deck'].fillna(0)
    dataset['Deck'] = dataset['Deck'].astype(int)
#dropping the cabin feature
train_df = train_df.drop(['Cabin'], axis=1)
test_df = test_df.drop(['Cabin'], axis=1)

In [ ]:
#filling the holes in age by random average values
data = [train_df, test_df]

for dataset in data:
    mean = train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)

In [ ]:
#filling the only couple of embarked missing value with the most common one
common_value = 'S'
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(common_value)

In [ ]:
#changing the float values of fare to int
for dataset in data:
    dataset['Fare'] = dataset['Fare'].fillna(0)
    dataset['Fare'] = dataset['Fare'].astype(int)

In [ ]:
#converting the sex feature into numeric
genders = {"male": 0, "female": 1}
data = [train_df, test_df]

for dataset in data:
    dataset['Sex'] = dataset['Sex'].map(genders)

In [ ]:
#converting embarked feature into numeric as well
ports = {"S": 0, "C": 1, "Q": 2}
data = [train_df, test_df]

for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].map(ports)

In [ ]:
#Categorizing age feature
data = [train_df, test_df]
for dataset in data:

    dataset.loc[ dataset['Age'] <= 5, 'Age'] = 0 #little children
    dataset.loc[(dataset['Age'] > 5) & (dataset['Age'] <= 14), 'Age'] = 1 #elemantary school to kousen
    dataset.loc[(dataset['Age'] > 15) & (dataset['Age'] <= 23), 'Age'] = 2 #kousen and university students
    dataset.loc[(dataset['Age'] > 23) & (dataset['Age'] <= 27), 'Age'] = 3 #社会人
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4 #marriage
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5 #creation of children
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6 #grandparents
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6 #waiting for death
train_df['Age'].value_counts()

In [ ]:
#Categorizing Fare feature
data = [train_df, test_df]

for dataset in data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0 #poor class
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1 #little poor class
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2 #middle class
    dataset.loc[(dataset['Fare'] > 31) & (dataset['Fare'] <= 99), 'Fare']   = 3 #rich middle class
    dataset.loc[(dataset['Fare'] > 99) & (dataset['Fare'] <= 250), 'Fare']   = 4 #rich class
    dataset.loc[ dataset['Fare'] > 250, 'Fare'] = 5 #millionaire++ class
    dataset['Fare'] = dataset['Fare'].astype(int)
train_df['Fare'].value_counts()

In [ ]:
#New features: FamiMem = number of relatives on the ship
data = [train_df, test_df]
for dataset in data:

    dataset['FamiMem'] = dataset['SibSp'] + dataset['Parch']
dataset['FamiMem'].astype(int)

train_df


In [ ]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

# Random Forest Model

In [ ]:
#random_forest = RandomForestClassifier(n_estimators=100)
#random_forest.fit(X_train, Y_train)

#Y_prediction = random_forest.predict(X_test)

#random_forest.score(X_train, Y_train)
#acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
#acc_random_forest

# Decision Tree Model

In [ ]:
#decision_tree = DecisionTreeClassifier()
#decision_tree.fit(X_train, Y_train)
#Y_prediction = decision_tree.predict(X_test)
#acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
#acc_decision_tree

# Logistic Regression Model

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

Y_prediction = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)

# Stochastic Gradient Descent

In [ ]:
#sgd = linear_model.SGDClassifier(max_iter=5, tol=None)
#sgd.fit(X_train, Y_train)
#Y_prediction = sgd.predict(X_test)

#sgd.score(X_train, Y_train)

#acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
#acc_sgd

### Conversion and Submission of the CSV file

In [ ]:
df_submission = pd.read_csv("../input/titanic/gender_submission.csv")
df_submission.drop('Survived', inplace=True, axis=1)

df = pd.DataFrame(Y_prediction)
df.columns = ['Survived']

df_submission = pd.concat([df_submission, df], axis=1)

df_submission.to_csv('submission.csv', index=False)

df_submission
